In [1]:
import pythran
%load_ext pythran.magic

In [2]:
def get_complex_random_array(dimensions):
    import numpy as np
    real_part = np.random.rand(*dimensions)
    imaginary_part = np.random.rand(*dimensions)
    complex_array = real_part + imaginary_part * 1.0j
    return complex_array

In [3]:
%%pythran -fopenmp -Ofast
#pythran export pythran_trapz_3dim_array(complex64[:, :, :], float64[:])

def pythran_trapz_3dim_array(y, x):
    import numpy as np 
    shape = y.shape
    result = np.zeros((shape[0], shape[1]), dtype=type(y[0, 0, 0]))
    "omp parallel for"
    for i in range(shape[0]):
        for j in range(shape[1]):
            out = .0 + .0j
            for k in range(len(y[i, j]) - 1):
                out += (x[k+1]-x[k]) * (y[i, j, k+1] + y[i, j, k])/2.0
            result[i, j] = out

    return result

In [4]:
import numpy as np
from numba import complex64, prange, jit, types

@jit(['complex64[:, :](complex64[:, :, :], float64[:])', 
        'complex128[:, :](complex128[:, :, :], float64[:])'],
     nopython=True, cache=True, nogil=True, parallel=True, fastmath=True)
def numba_trapz_3dim_array(y, x):
    shape = y.shape
    result = np.zeros((shape[0], shape[1]), dtype=type(y[0, 0, 0]))
    for i in prange(shape[0]):
        for j in range(shape[1]):
            out = .0 + .0j
            for k in range(len(y[i, j]) - 1):
                out += (x[k+1]-x[k]) * (y[i, j, k+1] + y[i, j, k])/2.0
            result[i, j] = out

    return result

In [5]:
def numpy_trapz_3dim_array(y, x):
    import numpy as np
    return np.trapz(y, x)

In [6]:
import datetime
import numpy as np

def test_trapz():
    numpy_time = []
    pythran_time = []
    numba_time = []
    for i in range(100):
        x = np.random.rand(710).astype(np.float64)
        y = get_complex_random_array((50, 300, 710)).astype(np.complex64)

        #numpy test zone
        numpy_start = datetime.datetime.now()
        res = numpy_trapz_3dim_array(y, x)
        numpy_end = datetime.datetime.now()
        numpy_time.append(numpy_end - numpy_start)

        #pythran test zone
        pythran_start = datetime.datetime.now()
        res = pythran_trapz_3dim_array(y, x)
        pythran_end = datetime.datetime.now()
        pythran_time.append(pythran_end - pythran_start)
        
        #numba test zone
        numba_start = datetime.datetime.now()
        res = numba_trapz_3dim_array(y, x)
        numba_end = datetime.datetime.now()
        numba_time.append(numba_end - numba_start)
        
    print('numpy time: ', np.sum(numpy_time))
    print('pythran time: ', np.sum(pythran_time))
    print('numba time: ', np.sum(numba_time))

    print('pythran speed up = ', np.sum(numpy_time) / np.sum(pythran_time))
    print('numba speed up = ', np.sum(numpy_time) / np.sum(numba_time))
    
    
test_trapz()

numpy time:  0:00:08.915986
pythran time:  0:00:00.440993
numba time:  0:00:00.373639
pythran speed up =  20.217976249056107
numba speed up =  23.86256787969136
